Data

In [54]:
!pip install pandas pycaret



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\student\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [55]:
# Import libraries
import pandas as pd
import requests
from pycaret.classification import *
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Fetch data from Air4Thai API
station_id = "44t"
param = "PM25,PM10,O3,CO,NO2,SO2,WS,TEMP,RH,WD,BP,RAIN"
data_type = "hr"

# Calculate start_date as 3 months ago from today
start_date = (datetime.today() - relativedelta(months=3)).strftime('%Y-%m-%d')

# Set end_date to today's date
end_date = datetime.today().strftime('%Y-%m-%d')

start_time = "00"
end_time = "23"

url = f"http://air4thai.com/forweb/getHistoryData.php?stationID={station_id}&param={param}&type={data_type}&sdate={start_date}&edate={end_date}&stime={start_time}&etime={end_time}"
response = requests.get(url)
response_json = response.json()
import pandas as pd

pd_from_dict = pd.DataFrame.from_dict(response_json["stations"][0]["data"])
csv_filename = f"air4thai_{station_id}_{start_date}_{end_date}.csv"
pd_from_dict.to_csv(csv_filename)


# Load your dataset containing PM25, PM10, O3, CO, NO2, SO2, WS, TEMP, RH, WD
data = pd.read_csv(f"air4thai_{station_id}_{start_date}_{end_date}.csv")

data.rename(columns={'TEMP': 'Temperature'}, inplace=True)
data.rename(columns={'WD': 'Wind Direction'}, inplace=True)
data.rename(columns={'RH': 'Relative Humidity'}, inplace=True)
data.rename(columns={'BP': 'Atmospheric Pressure'}, inplace=True)





In [56]:
import numpy as np
# Set the threshold for non-null values
train = data
threshold_percentage = 50
threshold = len(train) * (1 - threshold_percentage / 100)

# Drop columns with more than 50% NaN values
train.dropna(axis=1, thresh=threshold, inplace=True)

# Fill null values with the mean of each numeric column
numeric_columns = train.select_dtypes(include=[np.number]).columns
train[numeric_columns] = train[numeric_columns].fillna(train[numeric_columns].mean())

# Handle zeros separately
columns_to_handle_zeros = ["Temperature", "Relative Humidity", "Atmospheric Pressure", "Wind Direction"]

for column in columns_to_handle_zeros:
    train[column].replace(0, train[column].mean(), inplace=True)

print(train.describe())


       Unnamed: 0         PM25           O3           WS  Temperature  \
count  1701.00000  1701.000000  1701.000000  1701.000000  1701.000000   
mean    850.00000    18.256555    11.031099     0.904762    28.642633   
std     491.18072     5.488002    11.214633     0.455406     2.321996   
min       0.00000     5.300000     0.000000     0.000000    23.000000   
25%     425.00000    14.400000     2.000000     1.000000    27.000000   
50%     850.00000    17.800000    11.000000     1.000000    28.000000   
75%    1275.00000    21.300000    17.000000     1.000000    30.000000   
max    1700.00000    44.500000   140.000000     3.000000    36.000000   

       Relative Humidity  Wind Direction  Atmospheric Pressure         RAIN  
count        1701.000000     1701.000000           1701.000000  1701.000000  
mean           69.051057      135.599616            756.494514     0.048207  
std            11.764514       34.210581             11.375199     0.404422  
min            15.000000      

In [57]:
import jinja2

from pycaret.regression import *

session_1 = setup(data, target='PM25', train_size=0.8,
            numeric_features=['Relative Humidity', 'Wind Direction', 'Atmospheric Pressure', 'Temperature','RAIN'])

,Description,Value
0,Session id,6968
1,Target,PM25
2,Target type,Regression
3,Original data shape,"(1701, 10)"
4,Transformed data shape,"(1701, 10)"
5,Transformed train set shape,"(1360, 10)"
6,Transformed test set shape,"(341, 10)"
7,Numeric features,5
8,Categorical features,1
9,Preprocess,True


In [58]:
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,3.1327,17.0586,4.1223,0.4236,0.2134,0.1835,0.0130
en,Elastic Net,4.1154,28.3189,5.3109,0.0474,0.2842,0.2580,0.0100
lasso,Lasso Regression,4.1727,29.1910,5.3912,0.0188,0.2879,0.2613,0.0100
llar,Lasso Least Angle Regression,4.1727,29.1911,5.3912,0.0188,0.2879,0.2613,0.0100
omp,Orthogonal Matching Pursuit,4.1593,29.3654,5.4073,0.0124,0.2887,0.2608,0.0100
catboost,CatBoost Regressor,4.2248,29.8627,5.4527,-0.0035,0.2907,0.2641,0.3480
lightgbm,Light Gradient Boosting Machine,4.2268,29.9052,5.4567,-0.0050,0.2909,0.2641,0.0800
ridge,Ridge Regression,4.2288,29.9132,5.4574,-0.0053,0.2910,0.2644,0.0130
dt,Decision Tree Regressor,4.2286,29.9227,5.4582,-0.0055,0.2910,0.2642,0.0100
lr,Linear Regression,4.2296,29.9221,5.4582,-0.0056,0.2910,0.2644,0.0100


In [59]:
et = create_model('et')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.0448,27.8913,5.2812,-0.0010,0.2910,0.2628
1,4.3838,30.1635,5.4921,-0.0003,0.2888,0.2685
2,3.9736,25.4545,5.0452,-0.0199,0.2916,0.2716
3,4.1648,25.6956,5.0691,-0.0038,0.2727,0.2594
4,4.4890,34.2325,5.8509,-0.0092,0.3064,0.2728
5,4.0833,33.7069,5.8058,-0.0141,0.2821,0.2310
6,4.3596,31.4489,5.6079,-0.0021,0.2973,0.2751
7,3.9186,25.2068,5.0206,-0.0001,0.2641,0.2356
8,4.7027,37.4089,6.1163,-0.0024,0.3114,0.2838


In [60]:
# Manually define a parameter grid
param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20]}
tuned_et = tune_model(et, custom_grid=param_grid)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.0549,27.9336,5.2852,-0.0025,0.2916,0.2642
1,4.3892,30.2341,5.4986,-0.0027,0.2890,0.2685
2,3.9522,25.2785,5.0278,-0.0129,0.2901,0.2690
3,4.1667,25.7073,5.0702,-0.0042,0.2727,0.2595
4,4.4904,34.2296,5.8506,-0.0092,0.3065,0.2732
5,4.0850,33.7837,5.8124,-0.0164,0.2823,0.2308
6,4.3651,31.4733,5.6101,-0.0028,0.2977,0.2758
7,3.9199,25.2427,5.0242,-0.0015,0.2640,0.2351
8,4.7020,37.3895,6.1147,-0.0019,0.3113,0.2838


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 9 candidates, totalling 90 fits


Predict Model

In [61]:
predict_new = predict_model(tuned_et, data=train)
predict_new.describe()

data = predict_new
print(data.columns)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.8966,6.2192,2.4938,0.7934,0.1247,0.0517


Index(['Unnamed: 0', 'DATETIMEDATA', 'O3', 'WS', 'Temperature',
       'Relative Humidity', 'Wind Direction', 'Atmospheric Pressure', 'RAIN',
       'PM25', 'prediction_label'],
      dtype='object')


In [62]:

!pip install jupyter_dash


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\student\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [63]:


import webbrowser
import datetime
import pandas as pd
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html
from dash.dependencies import Output, Input

# Assuming 'train' is your DataFrame, modify this accordingly
train = predict_new

# Load your dataset containing PM25, PM10, O3, CO, NO2, SO2, WS, TEMP, RH, WD
# Comment out the line above and uncomment the line below if using a CSV file
# train = pd.read_csv("air4god_data.csv")

# Convert "DATETIMEDATA" to datetime format
train["DATETIMEDATA"] = pd.to_datetime(train["DATETIMEDATA"], format="%Y-%m-%d %H:%M:%S")
train.sort_values("DATETIMEDATA", inplace=True)

# Assuming 'prediction_label' is the column containing PM2.5 predictions in your DataFrame
pm25_predictions = train["prediction_label"].tolist()

# Create a DataFrame with hourly timestamps for the next 7 days
hours_in_a_day = 24
future_hourly_timestamps = [datetime.datetime.now() + datetime.timedelta(hours=i) for i in range(7 * hours_in_a_day)]
future_hourly_data = pd.DataFrame({"Timestamp": future_hourly_timestamps, "Predicted": pm25_predictions[:len(future_hourly_timestamps)]})

# Create Dash app
external_stylesheets = [
    {"href": "https://fonts.googleapis.com/css2?family=Lato:wght@400;700&display=swap", "rel": "stylesheet"},
]
app = Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Air Quality Analytics: Understand Your Air"

# Define the initial layout
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.P(children="🍃😎🍃", className="header-emoji"),
                html.H1(
                    children="Air Quality Analytics", className="header-title"
                ),
                html.P(
                    children="Analyze air quality parameters"
                    " including ,DATETIMEDATA,PM25,PM10,O3,CO,NO2,SO2,Wind Speed,Temperature,Relative Humidity,Wind Direction,Atmospheric Pressure,RAIN",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Parameter", className="menu-title"),
                        dcc.Dropdown(
                            id="parameter-filter",
                            options=[
                                {"label": param, "value": param}
                                for param in train.columns[2:]  # Skip the 'DATETIMEDATA' column
                            ],
                            value="PM25",  # Set default parameter
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(children="Date Range", className="menu-title"),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=train["DATETIMEDATA"].min().date(),
                            max_date_allowed=train["DATETIMEDATA"].max().date(),
                            start_date=train["DATETIMEDATA"].min().date(),
                            end_date=train["DATETIMEDATA"].max().date(),
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(children="Select Number of Days", className="menu-title"),
                        dcc.Slider(
                            id="day-slider",
                            min=1,
                            max=7,
                            step=1,
                            value=1,
                            marks={i: f"{i} {'' if i > 1 else ''}" for i in range(1, 8)},
                            className="slider",
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        dcc.Graph(
                            id="parameter-chart", config={"displayModeBar": False},
                        ),
                    ],
                    className="card",
                ),
                html.Div(
                    children=[
                        dcc.Graph(
                            id="pm25-predictions",
                            figure={
                                "data": [
                                    {"x": [], "y": [], "type": "line", "name": "PM2.5 Predictions"}
                                ],
                                "layout": {
                                    "title": "Hourly PM2.5 Predictions",
                                    "xaxis": {"title": "Timestamp"},
                                    "yaxis": {"title": "PM2.5 Predicted"},
                                },
                            },
                        ),
                    ],
                    className="card",
                ),
            ],
            className="wrapper",
        ),
    ]
)

# Define the first callback to update the parameter chart
@app.callback(
    Output("parameter-chart", "figure"),
    [
        Input("parameter-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ],
)
def update_chart(parameter, start_date, end_date):
    mask = (
        (train["DATETIMEDATA"] >= start_date)
        & (train["DATETIMEDATA"] <= end_date)
    )
    filtered_data = train.loc[mask, :]
    chart_figure = {
        "data": [
            {
                "x": filtered_data["DATETIMEDATA"],
                "y": filtered_data[parameter],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": f"{parameter} Levels", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#155d21"],
        },
    }
    return chart_figure

# Define the third callback to update the PM2.5 predictions chart
@app.callback(
    Output("pm25-predictions", "figure"),
    [
        Input("parameter-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
        Input("day-slider", "value"),
    ],
)
def update_pm25_predictions(parameter, start_date, end_date, selected_days):
    mask = (
        (train["DATETIMEDATA"] >= start_date)
        & (train["DATETIMEDATA"] <= end_date)
    )
    filtered_data = train.loc[mask, :]

    # Assuming 'prediction_label' is the column containing PM2.5 predictions in your DataFrame
    pm25_predictions = train["prediction_label"].tolist()

    # Create a DataFrame with hourly timestamps for the selected number of days
    hours_in_a_day = 24
    future_hourly_timestamps = [
        datetime.datetime.now() + datetime.timedelta(hours=i)
        for i in range(selected_days * hours_in_a_day)
    ]
    future_hourly_data = pd.DataFrame(
        {
            "Timestamp": future_hourly_timestamps,
            "Predicted": pm25_predictions[: len(future_hourly_timestamps)],
        }
    )

    figure = {
        "data": [
            {
                "x": future_hourly_data["Timestamp"],
                "y": future_hourly_data["Predicted"],
                "type": "line",
                "name": "PM2.5 Predictions",
            }
        ],
        "layout": {
            "title": {"text" :f"Hourly PM2.5 Predictions for the Next {selected_days} Days", "x": 0.05, "xanchor": "left"},
            "xaxis": {"title": "Timestamp"},
            "yaxis": {"title": "PM2.5 Predicted"},
            "colorway": ["#155d21"],
        },
    }

    return figure

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, mode='inline')

    # Open the app in a new browser window
    webbrowser.open_new_tab("http://127.0.0.1:8050/")
